<a href="https://colab.research.google.com/github/Hettieboo/Live_ShippingQuote_Calulator/blob/main/Shipping_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.3 MB/s eta 0:00:00


In [ ]:
"""
CONVELIO SHIPPING CALCULATOR - GRADIO APP
Single script that creates a web interface

Installation:
pip install gradio pandas openpyxl geopy reportlab

Usage for Google Colab:
1. Install: !pip install reportlab
2. Run the script
3. Upload your Excel file when prompted
4. App will launch with data pre-loaded

Usage for Local:
1. Install all dependencies
2. Place your Excel file in the same folder as this script
3. Run: python app.py
"""

import gradio as gr
import pandas as pd
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
import io

# Try to import reportlab for PDF generation
try:
    from reportlab.lib.pagesizes import letter
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.lib.units import inch
    from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
    from reportlab.lib import colors
    from reportlab.lib.enums import TA_CENTER, TA_RIGHT
    PDF_AVAILABLE = True
    print("✅ PDF generation available")
except ImportError:
    PDF_AVAILABLE = False
    print("⚠️ reportlab not installed - PDF download will be disabled")
    print("   Install with: pip install reportlab")

# ========== CONFIGURATION ==========
VALID_UNTIL_DATE = datetime(2025, 12, 8)

DELIVERY_TYPES = [
    'Front-delivery',
    'White Glove (ground floor)',
    'White Glove (with elevator)',
    'Curbside delivery'
]

PACKING_TYPES = [
    'Automatic (Convelio suggestion)',
    'Wood crate',
    'Cardboard box',
    'Bubble wrap only',
    'Custom packing'
]

# Initialize geocoder for address lookup
geolocator = Nominatim(user_agent="convelio_calculator_v1")

# Cache for geocoding results
geocode_cache = {}

# ========== LOAD EXCEL DATA ==========
lot_df = None

def load_excel_from_upload():
    """Load Excel file - for Colab or local"""
    global lot_df

    try:
        # Try Google Colab file upload
        from google.colab import files
        print("📁 Please upload your Excel file:")
        uploaded = files.upload()

        for filename in uploaded.keys():
            lot_df = pd.read_excel(io.BytesIO(uploaded[filename]))
            print(f"✅ Successfully loaded: {filename}")
            print(f"📊 Found {len(lot_df)} rows")
            print(f"📋 Columns: {', '.join(lot_df.columns)}")
            return True
    except ImportError:
        # Not in Colab, try loading from file
        try:
            EXCEL_FILE = "Calculateur _2302_Piasa_final_w. est. price_v.01.xlsx"
            lot_df = pd.read_excel(EXCEL_FILE)
            print(f"✅ Loaded {len(lot_df)} lots from {EXCEL_FILE}")
            print(f"📋 Columns: {', '.join(lot_df.columns)}")
            return True
        except Exception as e:
            print(f"❌ Error loading Excel file: {e}")
            print("⚠️ Please make sure your Excel file is in the same folder")
            return False

# Call this function before launching the app
print("="*50)
print("STEP 1: LOAD YOUR EXCEL FILE")
print("="*50)
load_excel_from_upload()

# ========== HELPER FUNCTIONS ==========

def calculate_days_remaining():
    """Calculate days until quote expires"""
    today = datetime.now()
    diff = (VALID_UNTIL_DATE - today).days
    return max(0, diff)

def search_address(query):
    """Search for addresses using Geopy"""
    if not query or len(query) < 3:
        return []

    # Check cache first
    if query in geocode_cache:
        return geocode_cache[query]

    try:
        locations = geolocator.geocode(query, exactly_one=False, limit=5, timeout=3)

        if locations:
            results = [loc.address for loc in locations]
            geocode_cache[query] = results
            return results
        return []
    except (GeocoderTimedOut, GeocoderServiceError):
        return []
    except Exception as e:
        print(f"Geocoding error: {e}")
        return []

def format_address(address_text):
    """Format and validate address"""
    if not address_text:
        return "Not specified"

    try:
        location = geolocator.geocode(address_text, timeout=3)
        if location:
            return location.address
        return address_text
    except:
        return address_text

def lookup_multiple_lots(lot_numbers_text):
    """Look up multiple lots from comma-separated input"""
    if lot_df is None:
        return "⚠️ Please upload Excel file first", ""

    if not lot_numbers_text or lot_numbers_text.strip() == "":
        return "", ""

    # Parse comma-separated lot numbers
    lot_numbers = [num.strip() for num in lot_numbers_text.split(',') if num.strip()]

    if len(lot_numbers) > 10:
        return "❌ Maximum 10 lots allowed per quote", ""

    all_descriptions = []
    sale_numbers = set()

    for lot_num_str in lot_numbers:
        try:
            lot_num = int(float(lot_num_str))
            lot_row = lot_df[lot_df['LOT'] == lot_num]

            if not lot_row.empty:
                description = str(lot_row.iloc[0]['TYPESET'])
                sale_no = str(int(lot_row.iloc[0]['SALENO'])) if 'SALENO' in lot_df.columns else "N/A"

                all_descriptions.append(f"--- LOT {lot_num} ---\n{description}\n")
                sale_numbers.add(sale_no)
            else:
                all_descriptions.append(f"--- LOT {lot_num} ---\n❌ Not found in database\n")
        except Exception as e:
            all_descriptions.append(f"--- {lot_num_str} ---\n❌ Error: {str(e)}\n")

    combined_description = "\n".join(all_descriptions)
    combined_sale = ", ".join(sorted(sale_numbers)) if sale_numbers else ""

    return combined_description, combined_sale

def calculate_total(convelio_offer, insurance):
    """Calculate total with insurance"""
    try:
        offer = float(convelio_offer) if convelio_offer else 0
        ins = float(insurance) if insurance else 0
        return offer + ins
    except:
        return 0

def suggest_packing_type(description):
    """Suggest packing type based on artwork description"""
    if not description or description.strip() == "":
        return "Automatic (Convelio suggestion)", "Automatic - let Convelio assess best option"

    description_lower = description.lower()

    # Check for fragile items
    fragile_keywords = ['glass', 'ceramic', 'porcelain', 'fragile', 'caisson lumineux',
                        'light box', 'neon', 'installation', 'sculpture', 'bronze', 'marble']

    # Check for large/heavy items
    heavy_keywords = ['sculpture', 'bronze', 'marble', 'stone', 'metal', 'installation']

    # Check for delicate works on paper
    paper_keywords = ['paper', 'watercolor', 'gouache', 'drawing', 'print', 'etching',
                      'lithograph', 'photograph', 'photo', 'c-print', 'tirage']

    # Check for paintings
    painting_keywords = ['canvas', 'oil', 'acrylic', 'painting', 'huile', 'toile']

    # Decision logic
    if any(keyword in description_lower for keyword in fragile_keywords):
        if any(keyword in description_lower for keyword in heavy_keywords):
            return "Wood crate", "Wood crate - fragile/heavy (glass, light box, or sculpture)"
        else:
            return "Wood crate", "Wood crate - fragile items (glass, light box, or delicate materials)"

    elif any(keyword in description_lower for keyword in paper_keywords):
        return "Cardboard box", "Cardboard box - works on paper (photograph, print, or paper-based)"

    elif any(keyword in description_lower for keyword in painting_keywords):
        return "Automatic (Convelio suggestion)", "Automatic - canvas paintings (standard protection)"

    elif any(keyword in description_lower for keyword in heavy_keywords):
        return "Wood crate", "Wood crate - heavy sculptures or installations"

    else:
        return "Automatic (Convelio suggestion)", "Automatic - let Convelio assess best option"

def suggest_packing_for_multiple_lots(lot_numbers_text):
    """Analyze each lot and provide individual packing suggestions"""
    if lot_df is None or not lot_numbers_text or lot_numbers_text.strip() == "":
        return "Automatic (Convelio suggestion)", "ℹ️ Enter lot numbers to get intelligent packing suggestions"

    # Parse comma-separated lot numbers
    lot_numbers = [num.strip() for num in lot_numbers_text.split(',') if num.strip()]

    if len(lot_numbers) == 0:
        return "Automatic (Convelio suggestion)", "ℹ️ Enter lot numbers to get intelligent packing suggestions"

    suggestions_list = []
    packing_recommendations = {}

    for lot_num_str in lot_numbers:
        try:
            lot_num = int(float(lot_num_str))
            lot_row = lot_df[lot_df['LOT'] == lot_num]

            if not lot_row.empty:
                description = str(lot_row.iloc[0]['TYPESET'])
                suggested_packing, reason = suggest_packing_type(description)

                suggestions_list.append(f"**Lot {lot_num}:** {reason}")

                # Count recommendations
                if suggested_packing in packing_recommendations:
                    packing_recommendations[suggested_packing] += 1
                else:
                    packing_recommendations[suggested_packing] = 1
            else:
                suggestions_list.append(f"**Lot {lot_num}:** Not found in database")
        except:
            suggestions_list.append(f"**{lot_num_str}:** Invalid lot number")

    # Determine overall recommendation
    if packing_recommendations:
        # If all lots need the same packing, recommend that
        if len(packing_recommendations) == 1:
            overall_packing = list(packing_recommendations.keys())[0]
            overall_message = f"\n\n✅ **Overall Recommendation:** {overall_packing} (all lots require same packing)"
        else:
            # If mixed, recommend the most protective (Wood crate > Cardboard > Automatic)
            if "Wood crate" in packing_recommendations:
                overall_packing = "Wood crate"
                overall_message = f"\n\n⚠️ **Overall Recommendation:** {overall_packing} (most protective option for mixed lot types)"
            elif "Cardboard box" in packing_recommendations:
                overall_packing = "Cardboard box"
                overall_message = f"\n\n💡 **Overall Recommendation:** {overall_packing} (suitable for most items)"
            else:
                overall_packing = "Automatic (Convelio suggestion)"
                overall_message = f"\n\n💡 **Overall Recommendation:** {overall_packing} (let Convelio assess)"
    else:
        overall_packing = "Automatic (Convelio suggestion)"
        overall_message = ""

    full_suggestion = "💡 **Packing Suggestions by Lot:**\n\n" + "\n".join(suggestions_list) + overall_message

    return overall_packing, full_suggestion

def generate_recap(packing, delivery, location, sale_no, lot_count=1):
    """Generate offer recap summary"""
    days = calculate_days_remaining()

    # Format address nicely
    formatted_location = format_address(location) if location else "Not specified"

    recap = f"""
### 📋 OFFER RECAP

**Number of Lots:** {lot_count}
**Sale Number:** {sale_no if sale_no else 'N/A'}
**Packing:** {packing if packing else 'Not selected'}
**Delivery Type:** {delivery if delivery else 'Not selected'}
**Destination:** {formatted_location}

---

⏰ **Quote valid until:** {VALID_UNTIL_DATE.strftime('%B %d, %Y')}
**Days remaining:** {days} days
"""
    return recap

def process_quote(lot_numbers, location, packing, delivery, convelio_offer, insurance):
    """Main function that processes all inputs and returns outputs"""

    # Look up lot details (handles multiple lots)
    description, sale_no = lookup_multiple_lots(lot_numbers)

    # Count number of lots
    lot_count = len([n for n in lot_numbers.split(',') if n.strip()]) if lot_numbers else 0

    # Calculate total
    total = calculate_total(convelio_offer, insurance)

    # Generate recap with lot count
    recap = generate_recap(packing, delivery, location, sale_no, lot_count)

    # Get packing suggestions for all lots
    suggested_packing, suggestion_text = suggest_packing_for_multiple_lots(lot_numbers)

    # Return description, sale number, total, recap, and packing suggestion info
    return description, sale_no, total, recap, suggestion_text

def generate_pdf_quote(lot_numbers, description, sale_no, location, packing, delivery, convelio_offer, insurance):
    """Generate a PDF quote document"""

    if not PDF_AVAILABLE:
        return None

    # Create PDF in memory
    buffer = io.BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter, topMargin=0.5*inch, bottomMargin=0.5*inch)

    # Container for the 'Flowable' objects
    elements = []

    # Define styles
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        textColor=colors.HexColor('#1e40af'),
        spaceAfter=30,
        alignment=TA_CENTER
    )

    heading_style = ParagraphStyle(
        'CustomHeading',
        parent=styles['Heading2'],
        fontSize=14,
        textColor=colors.HexColor('#1e40af'),
        spaceAfter=12,
        spaceBefore=12
    )

    # Title
    elements.append(Paragraph("📦 CONVELIO SHIPPING QUOTE", title_style))
    elements.append(Spacer(1, 0.3*inch))

    # Quote Info
    quote_date = datetime.now().strftime('%B %d, %Y')
    valid_until = VALID_UNTIL_DATE.strftime('%B %d, %Y')

    info_data = [
        ['Quote Date:', quote_date],
        ['Valid Until:', valid_until],
        ['Sale Number:', sale_no if sale_no else 'N/A']
    ]

    info_table = Table(info_data, colWidths=[2*inch, 4*inch])
    info_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
        ('FONTNAME', (1, 0), (1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('TEXTCOLOR', (0, 0), (0, -1), colors.HexColor('#4b5563')),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ]))
    elements.append(info_table)
    elements.append(Spacer(1, 0.3*inch))

    # Lot Information
    elements.append(Paragraph("LOT INFORMATION", heading_style))
    lot_count = len([n for n in lot_numbers.split(',') if n.strip()]) if lot_numbers else 0
    elements.append(Paragraph(f"<b>Number of Lots:</b> {lot_count}", styles['Normal']))
    elements.append(Paragraph(f"<b>Lot Numbers:</b> {lot_numbers}", styles['Normal']))
    elements.append(Spacer(1, 0.1*inch))

    # Description (truncated for PDF)
    desc_text = description[:800] + "..." if len(description) > 800 else description
    desc_clean = desc_text.replace('\n', '<br/>')
    elements.append(Paragraph(f"<b>Description:</b><br/>{desc_clean}", styles['Normal']))
    elements.append(Spacer(1, 0.2*inch))

    # Shipment Details
    elements.append(Paragraph("SHIPMENT DETAILS", heading_style))
    shipment_data = [
        ['Delivery Location:', location if location else 'Not specified'],
        ['Packing Type:', packing if packing else 'Not selected'],
        ['Delivery Type:', delivery if delivery else 'Not selected']
    ]

    shipment_table = Table(shipment_data, colWidths=[2*inch, 4*inch])
    shipment_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
        ('FONTNAME', (1, 0), (1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('TEXTCOLOR', (0, 0), (0, -1), colors.HexColor('#4b5563')),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ]))
    elements.append(shipment_table)
    elements.append(Spacer(1, 0.3*inch))

    # Pricing
    elements.append(Paragraph("PRICING", heading_style))

    offer_val = float(convelio_offer) if convelio_offer else 0
    insurance_val = float(insurance) if insurance else 0
    total_val = offer_val + insurance_val

    pricing_data = [
        ['Convelio Offer:', f'€{offer_val:,.2f}'],
        ['Insurance:', f'€{insurance_val:,.2f}'],
        ['', ''],
        ['TOTAL:', f'€{total_val:,.2f}']
    ]

    pricing_table = Table(pricing_data, colWidths=[4*inch, 2*inch])
    pricing_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (0, 1), 'Helvetica-Bold'),
        ('FONTNAME', (1, 0), (1, 1), 'Helvetica'),
        ('FONTNAME', (0, 3), (-1, 3), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 11),
        ('FONTSIZE', (0, 3), (-1, 3), 14),
        ('TEXTCOLOR', (0, 0), (0, -1), colors.HexColor('#4b5563')),
        ('ALIGN', (1, 0), (1, -1), 'RIGHT'),
        ('LINEABOVE', (0, 3), (-1, 3), 2, colors.HexColor('#1e40af')),
        ('TEXTCOLOR', (0, 3), (-1, 3), colors.HexColor('#1e40af')),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ]))
    elements.append(pricing_table)
    elements.append(Spacer(1, 0.4*inch))

    # Footer
    footer_text = f"""
    <para align=center>
    <font size=8 color="#6b7280">
    This quote is valid until {valid_until}.<br/>
    Generated by Convelio Shipping Calculator<br/>
    Art moderne et contemporain
    </font>
    </para>
    """
    elements.append(Paragraph(footer_text, styles['Normal']))

    # Build PDF
    doc.build(elements)

    # Get PDF data
    pdf_data = buffer.getvalue()
    buffer.close()

    return pdf_data

# ========== GRADIO INTERFACE ==========

def create_interface():
    """Create the Gradio interface"""

    # Get sale number from first row if data is loaded
    sale_display = ""
    if lot_df is not None and len(lot_df) > 0 and 'SALENO' in lot_df.columns:
        # Get the first sale number (assuming all lots are from same sale)
        first_sale = lot_df['SALENO'].iloc[0]
        sale_display = f" - Sale {int(first_sale)}"

    with gr.Blocks(theme=gr.themes.Soft(), title="Convelio Calculator") as demo:

        # Header
        if lot_df is not None:
            gr.Markdown(
                f"""
                # 📦 Convelio Shipping Calculator
                ### Art moderne et contemporain{sale_display}

                ✅ **Excel file loaded:** {len(lot_df)} lots available
                """
            )
        else:
            gr.Markdown(
                """
                # 📦 Convelio Shipping Calculator
                ### Art moderne et contemporain

                ⚠️ **No Excel file loaded** - Please upload file before launching app
                """
            )

        # Countdown display
        days = calculate_days_remaining()
        gr.Markdown(
            f"""
            <div style='background: #fef3c7; border: 2px solid #f59e0b; padding: 15px;
                        border-radius: 8px; margin: 10px 0;'>
                <strong>⏰ Quote valid until {VALID_UNTIL_DATE.strftime('%B %d, %Y')}</strong><br>
                <span style='color: #92400e; font-size: 18px; font-weight: bold;'>
                    {days} days remaining
                </span>
            </div>
            """
        )

        with gr.Row():
            # LEFT COLUMN - INPUTS
            with gr.Column(scale=1):
                gr.Markdown("### 📦 Lot Information")

                lot_number = gr.Textbox(
                    label="Lot Numbers (up to 10)",
                    placeholder="Enter lot numbers separated by commas (e.g., 86, 87, 88)",
                    info="Enter up to 10 lot numbers. Descriptions will auto-populate from Excel.",
                    lines=2
                )

                sale_number_display = gr.Textbox(
                    label="Sale Number(s)",
                    interactive=False,
                    placeholder="Will auto-populate from Excel"
                )

                description = gr.Textbox(
                    label="Description(s)",
                    lines=12,
                    interactive=False,
                    placeholder="Descriptions will appear here..."
                )

                gr.Markdown("### 📍 Shipment Parameters")

                location = gr.Textbox(
                    label="Delivery Location",
                    placeholder="Start typing address (e.g., '123 Main St, New York' or 'Louvre Museum, Paris')",
                    info="Suggestions will appear as you type (min 3 characters)",
                    lines=3,
                    max_lines=5
                )

                address_suggestions = gr.Radio(
                    label="📍 Address Suggestions (select one)",
                    choices=[],
                    visible=False,
                    interactive=True
                )

                packing = gr.Radio(
                    choices=PACKING_TYPES,
                    label="Type of Packing",
                    value=PACKING_TYPES[0]
                )

                packing_suggestion = gr.Markdown(
                    value="ℹ️ Enter lot numbers to get intelligent packing suggestions",
                    visible=True
                )

                delivery = gr.Radio(
                    choices=DELIVERY_TYPES,
                    label="Type of Delivery",
                    value=DELIVERY_TYPES[0]
                )

            # RIGHT COLUMN - OUTPUTS
            with gr.Column(scale=1):
                gr.Markdown("### 💰 Convelio Offer")

                convelio_offer = gr.Number(
                    label="Convelio Offer (EUR)",
                    placeholder="Enter offer amount",
                    value=0
                )

                insurance = gr.Number(
                    label="Insurance (EUR)",
                    placeholder="Enter insurance amount",
                    value=0
                )

                total = gr.Number(
                    label="💵 TOTAL WITH INSURANCE (EUR)",
                    interactive=False,
                    value=0
                )

                gr.Markdown("---")

                recap = gr.Markdown(
                    value=generate_recap("", "", "", "")
                )

                gr.Markdown("---")

                # Download button (only show if PDF available)
                if PDF_AVAILABLE:
                    download_btn = gr.Button("📥 Download Quote as PDF", variant="secondary", size="lg")
                    download_file = gr.File(label="Your Quote PDF", visible=False)
                else:
                    gr.Markdown("⚠️ *PDF download unavailable - install reportlab to enable*")

        # Calculate button
        calculate_btn = gr.Button("🔄 Calculate Total & Update Recap", variant="primary", size="lg")

        # Address search functionality - suggestions appear below, don't modify input
        def handle_address_input(address_query):
            """Show address suggestions in radio buttons without modifying the input"""
            if not address_query or len(address_query) < 3:
                return gr.Radio(choices=[], visible=False)

            suggestions = search_address(address_query)
            if suggestions and len(suggestions) > 0:
                # Show suggestions but don't auto-select - let user choose
                return gr.Radio(choices=suggestions, visible=True)
            return gr.Radio(choices=[], visible=False)

        # Auto-search as user types (shows suggestions in radio buttons below)
        location.change(
            fn=handle_address_input,
            inputs=[location],
            outputs=[address_suggestions]
        )

        # When user selects a suggestion, update the location field
        def update_location_from_suggestion(selected_address):
            """Update location field when user clicks a suggestion"""
            if selected_address:
                return selected_address, gr.Radio(visible=False)  # Hide suggestions after selection
            return "", gr.Radio(visible=False)

        address_suggestions.change(
            fn=update_location_from_suggestion,
            inputs=[address_suggestions],
            outputs=[location, address_suggestions]
        )

        # Auto-update description and sale number when lot numbers change
        def handle_lot_change(lot_numbers_text):
            """Handle lot number changes and provide packing suggestions"""
            description, sale_no = lookup_multiple_lots(lot_numbers_text)
            suggested_packing, suggestion_text = suggest_packing_for_multiple_lots(lot_numbers_text)

            return description, sale_no, suggested_packing, suggestion_text

        lot_number.change(
            fn=handle_lot_change,
            inputs=[lot_number],
            outputs=[description, sale_number_display, packing, packing_suggestion]
        )

        # Update everything when calculate is clicked
        calculate_btn.click(
            fn=process_quote,
            inputs=[lot_number, location, packing, delivery, convelio_offer, insurance],
            outputs=[description, sale_number_display, total, recap, packing_suggestion]
        )

        # Also auto-calculate total when offer or insurance changes
        convelio_offer.change(
            fn=calculate_total,
            inputs=[convelio_offer, insurance],
            outputs=[total]
        )

        insurance.change(
            fn=calculate_total,
            inputs=[convelio_offer, insurance],
            outputs=[total]
        )

        # PDF Download functionality (only if reportlab is available)
        if PDF_AVAILABLE:
            def create_pdf_download(lot_numbers, description, sale_no, location, packing, delivery, convelio_offer, insurance):
                """Create PDF and prepare for download"""
                if not lot_numbers or not location:
                    gr.Warning("Please fill in lot numbers and location before downloading")
                    return gr.File(visible=False)

                try:
                    pdf_data = generate_pdf_quote(
                        lot_numbers, description, sale_no, location,
                        packing, delivery, convelio_offer, insurance
                    )

                    if pdf_data is None:
                        gr.Warning("Error generating PDF")
                        return gr.File(visible=False)

                    # Save to temporary file
                    filename = f"Convelio_Quote_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
                    filepath = f"/tmp/{filename}"

                    with open(filepath, 'wb') as f:
                        f.write(pdf_data)

                    return gr.File(value=filepath, visible=True)
                except Exception as e:
                    gr.Warning(f"Error creating PDF: {str(e)}")
                    return gr.File(visible=False)

            download_btn.click(
                fn=create_pdf_download,
                inputs=[lot_number, description, sale_number_display, location, packing, delivery, convelio_offer, insurance],
                outputs=[download_file]
            )

        # Footer
        gr.Markdown(
            """
            ---
            ### 📋 How to use:
            1. **Enter lot numbers** (comma-separated for multiple, e.g., "86, 87, 88") - up to 10 lots per quote
            2. **Descriptions auto-populate** from your Excel file
            3. **Search for delivery address** - suggestions appear as you type
            4. **Select packing and delivery options**
            5. **Enter pricing** and click Calculate to see total and recap

            Made with Gradio | Powered by OpenStreetMap
            """
        )

    return demo

# ========== LAUNCH APP ==========

if __name__ == "__main__":
    print("\n" + "="*50)
    print("STEP 2: LAUNCHING APP")
    print("="*50)

    if lot_df is None:
        print("⚠️ WARNING: No Excel data loaded!")
        print("The app will launch but lot lookup won't work.")
        print("Please load your Excel file first.\n")

    demo = create_interface()

    demo.launch(
        share=True,
        show_error=True,
        inline=False
    )

    print("\n" + "="*50)
    print("🚀 Convelio Calculator is running!")
    print("="*50)

✅ PDF generation available
STEP 1: LOAD YOUR EXCEL FILE
📁 Please upload your Excel file:


Saving auction_dimensions_one_row_per_lot (23).xlsx to auction_dimensions_one_row_per_lot (23).xlsx
✅ Successfully loaded: auction_dimensions_one_row_per_lot (23).xlsx
📊 Found 86 rows
📋 Columns: SALENO, LOT, TYPESET, LOW, HIGH, ITEM_TYPE, ITEM_COUNT, MANUAL_REVIEW_REQUIRED, MATERIAL, PROCESSING_FLAGS, CONVERSION_LOG, D_1, Diameter_1, H_1, L_1, P_1, Diameter_2, D_2, H_2, L_2, P_2, Diameter_3, D_3, H_3, L_3, P_3, Diameter_4, D_4, H_4, L_4, P_4, Diameter_5, D_5, H_5, L_5, P_5, Diameter_6, D_6, H_6, L_6, P_6

STEP 2: LAUNCHING APP
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57939f5ccc3f70efe8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)

🚀 Convelio Calculator is running!
